### Trajectory Simulator for Ball and Goal from the 2020 & 2021 FRC Seasons
Interactive simulation for showing the ball trajectory launched toward the Inner & Outer Goals

#### Condition of the ball as launched is defined by 4 parameters:
1) X0:    Horizontal distance [m] from the Outer Goal; Outer Goal is located at X=0,so X will be negative.
2) Y0:    Verticle distance from the ground [m]
3) Speed: Speed of the ball [m/s]
4) Angle: Angle relative to the ground [deg]

#### During flight, the ball is acted on by 2 forces:
1) Gravity: Acts downward, with a constant value of g = 9.805 [m/s^2]
2) Drag from air resistance:
   a) Magnitude of the drag force is proportional to the speed squared
   b) Direction of the drag force is opposite to the direction of travel

#### The rectangles on the plot indicate the locations of the Outer and Inner goals.
1) To go through the Inner Goal, the ball has to also go through the Outer Goal
2) The height of the rectangles indicate the height of the opening, while the trajectory line shows where the center of the ball will travel, so if the line is close to the edge of the goal it may get deflected instead of going through.

#### Interactive control
The 4 sliders are for controlling the initial launch conditions.
Various combinations can be tried to see:
1) What conditions are viable
2) What conditions are more robust
    Example: What combnation of Speed & Angle will score a goal over a wider range of X0 values



In [1]:
import numpy as np
import scipy as sp
from scipy.integrate import solve_ivp
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Rectangle
from matplotlib.collections import PatchCollection
import ipywidgets as widgets

In [2]:
# Function describing the set of equations of motion for a ball traveling with air resistance
def dSdt(t, S, alpha, mass):
    # Coupled differential equation for the trajectory of a ball
    # Assumes the ball is lauched with some initial speed and some angle relative to the ground
    # Model accouts for the force of gravity and drag due to air resistance
    #
    # x & y are locations; units: [m]
    #
    # For differential equations with respect to time, we are using "dot notation".
    #    x_dot = dx/dt, which is also the component of velocity in the X-direction: vx
    #    y_dot = dy/dt, which is also the component of velocity in the Y-direction: vy
    #
    #    vx_dot = dvx/dt, which is also called the acceleration in the X-direction
    #    vy_dot = dvy/dt, which is also called the acceleration in the Y-direction
    #
    #    When defining the set of 4 differential equations, we use the dot notation.
    #    But, we use vx and vy, instead of x_dot and y_dot, when referring to the variables for calculations
    #
    # INPUTS:  t:     time (note, not use by this function but include as a standard format when using solve_ivp)
    #          S:     set of equations (column for each variable in the coupled set of equations)
    #          alpha: parameter for calculating the drag force due to air resistance; units: [kg/m]
    #          mass:  mass of the ball; units: [kg]
    # OUTPUTS: x_dot: the updated dx/dt value
    #          vx_dot: the updated dvx/dt value
    #          y_dot: the updated dy/dt value
    #          vy_dot: the updated dvy/dt value
    #
    #  Model: F_net = F_grav + F_drag
    #         Where: F_grav = -g in the Y-direction
    #             F_drag = alpha * v^2 in the opposite direction of travel
    #             This can be expressed as:
    #                 F_drag = -alpha * |v| * v
    #                     This form of expression gives:
    #                          Magnitude: alpha * v^2
    #                          Direction: opposite to the direction of travel
    #         The acceleration of the ball is determined by the net force on the ball
    #             ma = F_net, so a = F_net / m
    #             Using dot notation for the components in the X & Y diraction:
    #               ax = vx_dot = -(alpha/m)*|v|*vx
    #               ay = vy_dot = -g -(alpha/m)*|v|*vy
 
    x, vx, y, vy = S
    g = 9.805
    q = (alpha / mass) * np.sqrt(vx**2 + vy**2)
    x_dot = vx
    vx_dot = -q * vx
    y_dot = vy
    vy_dot = -g -(q * vy)
    return [ x_dot, vx_dot, y_dot, vy_dot]


In [3]:
# Determine various values for the yellow ball used in the 2020 & 2021 FRC seasons
ball_mass = 0.142     # mass in kg

# alpha = 0.5 * air_density * cross-sectional_area * Cd
air_density = 1.225                # kg/m^3; value for sea level at 15C
ball_radius = (7/2)*(2.54/100)     # 7 inch diameter converted to radius in meters
Cd = 0.5                           # Estimate for spherical object
alpha = 0.5 * air_density * 3.14159 * (ball_radius**2) * Cd

In [16]:
# Define the default figure size
plt.rcParams['figure.figsize'] = [20, 20]

In [17]:
# Define input sliders
speed = widgets.FloatSlider(value=10, min=0, max=15, readout_format='.1f', description='Speed:')
angle = widgets.FloatSlider(value=45, min=20, max=80, readout_format='.1f', description='Angle:')
x0 = widgets.FloatSlider(value=-5, min=-8, max=0, readout_format='.1f', description='X0:')
y0 = widgets.FloatSlider(value=0.5, min=0.2, max=1.0, readout_format='.1f', description='Y0:')

# Control layout of sliders
ui = widgets.VBox([speed, angle, x0, y0])

# Function for analysis based on initial conditions
def plot(x_init=-5, y_init=0.5, speed_init=10, angle_init=45):
    x_initial = x_init
    y_initial = y_init

    launch_angle_rad = np.deg2rad(angle_init)
    vx_initial = speed_init * np.cos(launch_angle_rad)
    vy_initial = speed_init * np.sin(launch_angle_rad)

    iValues = [x_initial, vx_initial, y_initial, vy_initial]

    # Find solution
    sol = solve_ivp(dSdt, [0,2], y0=iValues, t_eval=np.linspace(0,2,1000), args=(alpha, ball_mass))

    # Plot
    #plt.figure(figsize=(10, 10))
    fig, ax = plt.subplots()
    #ax.set_aspect('equal', 'box')
    ax.set_aspect('equal')

    plt.plot(sol.y[0], sol.y[2], color='green')

    patches = []
    alphas = []
    outerGoal = Rectangle((0, 2.50 - (0.76 / 2)), 0.05, 0.76)
    patches.append(outerGoal)
    alphas.append(0.3)
    
    innerGoal = Rectangle((0.74, 2.50 - (0.33 / 2)), 0.05, 0.33)
    patches.append(innerGoal)
    alphas.append(0.3)


    p = PatchCollection(patches, alpha=alphas)
    ax.add_collection(p)

    plt.xlim(left = -8, right=2)
    plt.ylim(bottom=0, top=3)

    plt.show()

#####
out = widgets.interactive_output(plot, {'x_init': x0, 'y_init': y0, 'speed_init': speed, 'angle_init': angle})

display(out, ui)


Output()